In [2]:
import datetime
import os

from dateutil.relativedelta import relativedelta

import camelot
import dateparser
import magic
import pandas as pd
import requests
import tabula

from bs4 import BeautifulSoup

In [3]:
!pwd

/home/sean/Projects/housing/notebooks


In [5]:
os.chdir('../data/raw')
!ls

RAENewsRelease2-Feb2019.pdf  RAENewsRelease-Jun2019.pdf
RAENewsRelease-Apr2018.pdf   RAENewsRelease-June2018.pdf
RAENewsRelease-Apr2019.pdf   RAENewsRelease-Mar2018.pdf
RAENewsRelease-Aug2018.pdf   RAENewsRelease-Mar2019.pdf
RAENewsRelease-Dec2018.pdf   RAENewsRelease-May2018.pdf
RAENewsRelease-Feb2018.pdf   RAENewsRelease-May2019.pdf
RAENewsRelease-Jan2018.pdf   RAENewsRelease-Nov2018.pdf
RAENewsRelease-Jan2019.pdf   RAENewsRelease-Oct2018.pdf
RAENewsRelease-July2018.pdf  RAENewsRelease-Sept2018.pdf


In [6]:
link_page = "https://realtorsofedmonton.com/web/RAE_Public/Market_Stats/Monthly%20Market%20Statistics/RAE_Public/Market_Statistics/Monthly_Market_Stats.aspx?hkey=ab27cbda-7c81-4393-8a5f-aaf1277eba97"
page = requests.get(link_page)
soup = BeautifulSoup(page.content, 'html.parser')

pdf_links = {link['href'] for link in soup.find_all('a')
             if ('href' in link.attrs
                 and 'RAE' in link['href']
                 and 'pdf' == link['href'][-3:])
            }

In [12]:
pdfs = !ls

for pdf in pdf_links:
    file = pdf.split('/')[-1]
    if file not in pdfs:
        print(pdf)
        #run once: !wget $pdf ./

https://member.ereb.com/WEB/Documents/pdf/YEGmedia-RAEstatsMay2019.pdf


In [ ]:
pdfs = !ls

# fix errors 

for pdf in pdfs:
    if 'HTML' in magic.from_file(pdf):
        date = pdf.split('-')[1][:-4]
        date_time = dateparser.parse(date)
        date_time -= relativedelta(months=1)
        date = date_time.strftime('%b%Y')
        link = 'https://member.ereb.com/WEB/Documents/pdf/'
        alt_format = f'YEGmedia-RAEstats{date}.pdf'
        pdf_link = link + alt_format
        print(alt_format)
        
        # wget alternative format
        !wget $pdf_link -O ./$pdf

    elif 'YEG' in pdf:
        date = pdf.split('-')[1][:-4]
        date = date.replace('RAEstats', '')
        date_time = dateparser.parse(date)
        date_time += relativedelta(months=1)
        date = date_time.strftime('%b%Y')
        alt_format = f'RAENewsRelease-{date}.pdf'
        print(alt_format)

        # rename file to match other formats
        !mv $pdf $alt_format
 